In [1]:
import numpy as np

In [295]:
c=np.array([3,2])
A=np.array([[1,-1],[3,1],[4,3]])
b=np.array([2,5,7])

# Problem 1+2+3+4+5+6

In [298]:
#Problem 1: define the class:
class simplex(object):
    def __init__(self, c,A,b):
        self.c=c
        self.A=A
        self.b=b
        self.origin_fsb=np.all(np.dot(A,np.zeros(len(A[0,:])))<=b)
        if self.origin_fsb==False:
            raise ValueError("The problem is not feasible at the origin")
#Problem 2: create slack variables and add the store method:
    def basicstore(self):
        self.orig=np.zeros(len(self.c))
        m=len(A[:,0])
        n=len(A[0,:])
        self.slack=self.b-np.dot(self.A,self.orig)
        vrbs=np.append(self.orig, self.slack)
        L=np.append(np.where(vrbs!=0)[0],np.where(vrbs==0)[0])
        return L
#Problem 3: create the initial tableau:
    def init_tableau(self):
        m=len(self.A[:,0])
        I=np.eye(m)
        Abar=np.hstack((self.A, I))
        cbar=np.append(self.c, np.zeros(m))
        return np.vstack((np.hstack(([0],-cbar,[1])),np.hstack((np.array([b]).T,Abar,
                                                                          np.array([np.zeros(len(b))]).T))))
#Problem 4: return the pivot indices
    def pivotind(self, tableau):
        self.tableau=tableau
        T=self.tableau
        negindex=np.where(T[0,:]<0)[0]
        enter=negindex.min()
        pos_enter=np.where(T[:,enter]>0)[0]
        if len(pos_enter)==0:
            raise ValueError('The problem is unbounded')
        else:
            ratio=T[:,0][pos_enter]/T[:,enter][pos_enter]
            minratio=ratio.min()
            minloc=np.where(ratio==minratio)[0]
            leave=np.where(((T[:,0]==T[:,0][pos_enter][minloc]).astype(int))*((T[:,enter]==
                        T[:,enter][pos_enter][minloc]).astype(int))==1)[0]
        return enter, leave
#Problem 5: perform a single pivot to completion:
    def pivot(self, tableau, L):
        self.tableau=tableau
        self.orig=np.zeros(len(self.c))
        T=self.tableau
        enter, leave=self.pivotind(T)
        L[np.where(L==(enter-1))[0]],L[leave-1]=L[leave-1],L[np.where(L==(enter-1))[0]]
        T[leave,:]=T[leave,:]/T[leave,enter]
        for i in range(len(T[:,0])):
            T[i,:]=T[i,:]-(T[leave,:]*T[i,enter])*(1-int(i==leave))
        return T, L
#Problem 6: solve
    def solve(self):
        self.tableau=self.init_tableau()
        orig=np.array([0,0])
        L=self.basicstore()
        T=self.tableau
        while (len(np.where(T[0,:]<0)[0])>0):
            T,L=self.pivot(T,L)
        return T, L
            
            

In [299]:
problem=simplex(c, A, b)
problem.solve()

(array([[ 5.2,  0. ,  0. ,  0. ,  0.2,  0.6,  1. ],
        [ 0.6,  0. ,  0. ,  1. , -1.4,  0.8,  0. ],
        [ 1.6,  1. ,  0. ,  0. ,  0.6, -0.2,  0. ],
        [ 0.2,  0. ,  1. ,  0. , -0.8,  0.6,  0. ]]), array([2, 0, 1, 3, 4]))

# Problem 7

In [300]:
product=np.load('productMix.npz')
A=product['A']
p=product['p']
m=product['m']
d=product['d']
A,p,m,d

(array([[ 0.12,  0.18,  0.13,  0.07],
        [ 0.15,  0.12,  0.13,  0.11],
        [ 0.1 ,  0.1 ,  0.1 ,  0.12]]),
 array([ 250.,  215.,  275.,  180.]),
 array([ 4.,  4.,  4.]),
 array([ 10.,  20.,  12.,  10.]))

In [304]:
Abar=np.vstack((A,np.eye(4)))
b=np.append(m,d)
prob7=simplex(p,Abar,b)
sol,lis=prob7.solve()
sol[:,0][lis<4]


/Users/hqdhftw/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 8 but corresponding boolean dimension is 11
  """


(array([[  7.45359649e+03,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   1.79824561e+02,
           1.52192982e+03,   0.00000000e+00,   1.31578947e-01,
           0.00000000e+00,   5.37719298e+01,   0.00000000e+00,
           1.00000000e+00],
        [  6.19298246e+00,   0.00000000e+00,   1.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   9.64912281e+00,
          -6.14035088e+00,   0.00000000e+00,  -2.36842105e-01,
           0.00000000e+00,  -4.56140351e-01,   0.00000000e+00,
           0.00000000e+00],
        [  1.78947368e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   1.00000000e+00,  -1.05263158e+01,
           1.57894737e+01,   0.00000000e+00,  -1.10526316e+00,
           0.00000000e+00,  -6.84210526e-01,   0.00000000e+00,
           0.00000000e+00],
        [  9.65964912e-01,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   2.98245614e-01,
          -1.28070175e+00,   1.000